In [ ]:
!pip install setuptools


In [ ]:
!pip install pandas_datareader


In [ ]:
!pip install yfinance


In [ ]:
import yfinance as yf
import pandas_datareader as pdr
import pandas as pd

# Desired stock

In [5]:
#define my ticket symmbol
tickerSymbol = 'KO'

In [6]:
# Get data for the ticker
tickerData = yf.Ticker(tickerSymbol)

# Get the historical prices for the ticker
stock = tickerData.history(start='1990-01-01', end='2022-01-01')

# Select the 'Close' price column
stock = stock[['Close']]

# Reset the index to convert the date index into a column
stock.reset_index(inplace=True)

# Format the date column to yyyymmdd
stock['Date'] = stock['Date'].dt.strftime('%Y-%m-%d')

# Rename the index from 'Date' to 'DATE'
stock.rename(columns={'Date': 'DATE'}, inplace=True)
stock.rename(columns={'Close': 'Returns'}, inplace=True)

# See the data
stock.tail()

,DATE,Returns
8059,2021-12-27,53.585861
8060,2021-12-28,53.796001
8061,2021-12-29,53.859947
8062,2021-12-30,53.704632
8063,2021-12-31,54.097496


# Unadjusted GDP

In [7]:
import pandas_datareader.data as pdr
from datetime import datetime

# Define the time range
start_date = datetime(1990, 1, 1)  # Starting from January 1, 2000
end_date = datetime(2022,1,1)          # Until today's date

In [8]:
# Fetch GDP data from FRED
gdp_data = pdr.DataReader('GDP', 'fred', start_date, end_date)


# Display the data
gdp_data.head()

,GDP
DATE,
1990-01-01,5872.701
1990-04-01,5960.028
1990-07-01,6015.116
1990-10-01,6004.733
1991-01-01,6035.178


# Unemployement 

In [9]:
#Fetch unemployment rate data from FRED
unemployment_data = pdr.DataReader('UNRATE', 'fred', start_date, end_date)



# Rename the column 
unemployment_data.rename(columns={'UNRATE': 'Unemployment_Rate'}, inplace=True)

# Display the DataFrame
unemployment_data.head()

,Unemployment_Rate
DATE,
1990-01-01,5.4
1990-02-01,5.3
1990-03-01,5.2
1990-04-01,5.4
1990-05-01,5.4


# Exchange Rate USD to Euro

In [10]:
# Fetch exchange rate data from FRED
exchange_rate_data = pdr.DataReader('DEXUSEU', 'fred', start_date, end_date)

# Rename the column 
exchange_rate_data.rename(columns={'DEXUSEU': 'Exchange_Rate(USD to Euro)'}, inplace=True)


# Federal Funds Rate

In [11]:
# Fetch federal funds rate data from FRED
federal_funds_rate_data = pdr.DataReader('FEDFUNDS', 'fred', start_date, end_date)

# Rename the column for clarity (optional)
federal_funds_rate_data.rename(columns={'FEDFUNDS': 'Federal_Funds_Rate'}, inplace=True)

# Dummy Variable

In [12]:
import pandas as pd

# Create a date range from 1990-01-01 to 2021-01-01 with monthly frequency
dates = pd.date_range(start="1990-01-01", end="2022-01-01", freq="MS")

# Years to assign the dummy variable as 1
dummy_years = [1994, 1996, 1999, 2001, 2007, 2008, 2009, 2010, 2011,2018,2020,2022]

# Create a data frame with the date and dummy variable
dummy_df = pd.DataFrame({
    "DATE": dates,
    "Dummy": [1 if date.year in dummy_years else 0 for date in dates]
})
dummy_df


,DATE,Dummy
0,1990-01-01,0
1,1990-02-01,0
2,1990-03-01,0
3,1990-04-01,0
4,1990-05-01,0
...,...,...
380,2021-09-01,0
381,2021-10-01,0
382,2021-11-01,0
383,2021-12-01,0


# VIX Yfinance

In [13]:
import yfinance as yf

# Define the ticker symbol
tickerSymbol = '^VIX'

# Get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

# Get the historical prices for this ticker
tickerDF = tickerData.history(start='1990-01-01', end='2022-01-01')

# Keep only the relevant columns
tickerDF = tickerDF[['Close', 'Volume', 'Dividends', 'Stock Splits']]

# Reset the index to convert the date index into a column
tickerDF.reset_index(inplace=True)

# Format the date column to yyyymmdd
tickerDF['Date'] = tickerDF['Date'].dt.strftime('%Y-%m-%d')

# Rename the index from 'Date' to 'DATE'
tickerDF.rename(columns={'Date': 'DATE'}, inplace=True)
tickerDF.rename(columns={'Close': 'VIX'}, inplace=True)

VixClose = tickerDF.iloc[:,:2]
VixClose


,DATE,VIX
0,1990-01-02,17.240000
1,1990-01-03,18.190001
2,1990-01-04,19.219999
3,1990-01-05,20.110001
4,1990-01-08,20.260000
...,...,...
8059,2021-12-27,17.680000
8060,2021-12-28,17.540001
8061,2021-12-29,16.950001
8062,2021-12-30,17.330000


# Join

In [14]:
merged_data = pd.merge(gdp_data, unemployment_data, on='DATE', how='outer')

In [15]:
merged_data = pd.merge(merged_data, exchange_rate_data, on='DATE', how='outer')

In [16]:
merged_data = pd.merge(merged_data, federal_funds_rate_data, on='DATE', how='outer')

In [17]:
#merged_data = pd.merge(merged_data, dummy_df, on='DATE', how='outer')

In [18]:
merged_data.reset_index(inplace=True)
# Ensure both 'DATE' columns are datetime64 type
merged_data['DATE'] = pd.to_datetime(merged_data['DATE'])
VixClose['DATE'] = pd.to_datetime(VixClose['DATE'])

# Now perform the merge
merged_data = pd.merge(merged_data, VixClose, on='DATE', how='outer')

In [19]:
merged_data

,DATE,GDP,Unemployment_Rate,Exchange_Rate(USD to Euro),Federal_Funds_Rate,VIX
0,1990-01-01,5872.701,5.4,NaN,8.23,NaN
1,1990-01-02,NaN,NaN,NaN,NaN,17.240000
2,1990-01-03,NaN,NaN,NaN,NaN,18.190001
3,1990-01-04,NaN,NaN,NaN,NaN,19.219999
4,1990-01-05,NaN,NaN,NaN,NaN,20.110001
...,...,...,...,...,...,...
8391,2021-12-28,NaN,NaN,1.1314,NaN,17.540001
8392,2021-12-29,NaN,NaN,1.1337,NaN,16.950001
8393,2021-12-30,NaN,NaN,1.1318,NaN,17.330000
8394,2021-12-31,NaN,NaN,NaN,NaN,17.219999


In [20]:
merged_data['GDP'] = merged_data['GDP'].fillna(method='ffill')
merged_data['Federal_Funds_Rate'] = merged_data['Federal_Funds_Rate'].fillna(method='ffill')
merged_data['Unemployment_Rate'] = merged_data['Unemployment_Rate'].fillna(method='ffill')

# Fill NaN values in the Exchange Rate column with 0
#merged_data['Exchange_Rate(USD to Euro)'] = merged_data['Exchange_Rate(USD to Euro)'].fillna(0)

merged_data

C:\Users\avira\AppData\Local\Temp\ipykernel_24856\3404549237.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data['GDP'] = merged_data['GDP'].fillna(method='ffill')
C:\Users\avira\AppData\Local\Temp\ipykernel_24856\3404549237.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data['Federal_Funds_Rate'] = merged_data['Federal_Funds_Rate'].fillna(method='ffill')
C:\Users\avira\AppData\Local\Temp\ipykernel_24856\3404549237.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data['Unemployment_Rate'] = merged_data['Unemployment_Rate'].fillna(method='ffill')


,DATE,GDP,Unemployment_Rate,Exchange_Rate(USD to Euro),Federal_Funds_Rate,VIX
0,1990-01-01,5872.701,5.4,NaN,8.23,NaN
1,1990-01-02,5872.701,5.4,NaN,8.23,17.240000
2,1990-01-03,5872.701,5.4,NaN,8.23,18.190001
3,1990-01-04,5872.701,5.4,NaN,8.23,19.219999
4,1990-01-05,5872.701,5.4,NaN,8.23,20.110001
...,...,...,...,...,...,...
8391,2021-12-28,24777.038,3.9,1.1314,0.08,17.540001
8392,2021-12-29,24777.038,3.9,1.1337,0.08,16.950001
8393,2021-12-30,24777.038,3.9,1.1318,0.08,17.330000
8394,2021-12-31,24777.038,3.9,NaN,0.08,17.219999


# Joining Stock data

In [21]:

stock['DATE'] = pd.to_datetime(stock['DATE'])

# Now perform the merge
merged_data = pd.merge(merged_data, stock, on='DATE', how='outer')

In [22]:
merged_data

,DATE,GDP,Unemployment_Rate,Exchange_Rate(USD to Euro),Federal_Funds_Rate,VIX,Returns
0,1990-01-01,5872.701,5.4,NaN,8.23,NaN,NaN
1,1990-01-02,5872.701,5.4,NaN,8.23,17.240000,2.119077
2,1990-01-03,5872.701,5.4,NaN,8.23,18.190001,2.088513
3,1990-01-04,5872.701,5.4,NaN,8.23,19.219999,2.078326
4,1990-01-05,5872.701,5.4,NaN,8.23,20.110001,2.061346
...,...,...,...,...,...,...,...
8391,2021-12-28,24777.038,3.9,1.1314,0.08,17.540001,53.796001
8392,2021-12-29,24777.038,3.9,1.1337,0.08,16.950001,53.859947
8393,2021-12-30,24777.038,3.9,1.1318,0.08,17.330000,53.704632
8394,2021-12-31,24777.038,3.9,NaN,0.08,17.219999,54.097496


In [25]:
# Save the file to the desired location
save_path = r"C:\Users\avira\Downloads\stock_prediction_richter\combined.csv"
merged_data.to_csv(save_path, index=False)

print(f"File saved to: {save_path}")


File saved to: C:\Users\avira\Downloads\stock_prediction_richter\combined.csv
